In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
import seaborn as sns
from collections import deque
import os
import gc
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [ ]:
# f = open('./data/tr_j.bin', 'rb')
# tr_j = pk.load(f)
# f.close()
# f = open('./data/te_j.bin', 'rb')
# te_j = pk.load(f)
# f.close()

In [ ]:
%%time
# data load
f = open('./data/train.bin', 'rb')
tr = pk.load(f)
f.close
f = open('./data/test.bin', 'rb')
te = pk.load(f)
f.close()
sub = pd.read_csv('./data/sample_submission.csv')

tr['event_contents'] = tr['event_contents'].replace(np.nan, 'NA')
te['event_contents'] = te['event_contents'].replace(np.nan, 'NA')

target = tr.groupby('game_id').mean()['winner']


In [ ]:
tr_n = np.unique(tr['game_id'])
te_n = np.unique(te['game_id'])
temp_tr = tr[tr['event'] == 0]
temp_te = te[te['event'] == 0]

In [ ]:
%%time
# player's species
tr_q = tr.groupby(['game_id', 'player']).max()
tr_p = pd.DataFrame(columns=['p1', 'p2'])
ww = tr_q['species'].values
w0=[]
w1=[]
for i in range(len(tr_q['species'].values)//2):
    w0.append(ww[2*i]) 
    w1.append(ww[2*i+1])
tr_p['p1'] = w0
tr_p['p2'] = w1


te_q = te.groupby(['game_id', 'player']).max()
te_p = pd.DataFrame(columns=['p1', 'p2'])
ww = te_q['species'].values
w0=[]
w1=[]
for i in range(len(te_q['species'].values)//2):
    w0.append(ww[2*i]) 
    w1.append(ww[2*i+1])
te_p['p1'] = w0
te_p['p2'] = w1

In [ ]:
%%time
# count event rec <= apm proxy
tr_count = tr.groupby(['game_id', 'player'])['event'].count().values
te_count = te.groupby(['game_id', 'player'])['event'].count().values
w0=[]
w1=[]
for i in range(len(tr_count)//2):
    w0.append(tr_count[2*i]) 
    w1.append(tr_count[2*i+1])
tr_c = pd.DataFrame(columns=['apm'])
tr_c['apm'] = (np.array(w0)-np.array(w1)) / (np.array(w0)+np.array(w1))
tr_c['apm_p1'] = np.array(w0)
tr_c['apm_p2'] = np.array(w1)
tr_c['apm_avg1'] = np.array(w0)/(np.array(w0) + np.array(w1))
tr_c['apm_avg2'] = np.array(w1)/(np.array(w0) + np.array(w1))

w0=[]
w1=[]
for i in range(len(te_count)//2):
    w0.append(te_count[2*i]) 
    w1.append(te_count[2*i+1])
te_c = pd.DataFrame(columns=['apm'])
te_c['apm'] = (np.array(w0)-np.array(w1)) / (np.array(w0)+np.array(w1))
te_c['apm_p1'] = w0
te_c['apm_p2'] = w1
te_c['apm_avg1'] = np.array(w0)/(np.array(w0) + np.array(w1))
te_c['apm_avg2'] = np.array(w1)/(np.array(w0) + np.array(w1))


In [ ]:
# game time(sec)
tr_time = tr.groupby('game_id')['time'].max()

te_time = te.groupby('game_id')['time'].max()

# unit: extract cumulated amount

In [ ]:
# for train
w0m = [0] * tr_n
w0g = [0] * tr_n
w0c = [0] * tr_n
w1m = [0]* tr_n
w1g = [0] * tr_n
w1c = [0] * tr_n
for i in tqdm(range(len(temp_tr))):
    for k in Unit_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            if temp_tr['player'].values[i] == 0:
                w0m[temp_tr['game_id'].values[i]] += Unit_dict[k][0]
                w0g[temp_tr['game_id'].values[i]] += Unit_dict[k][1]
                w0c[temp_tr['game_id'].values[i]] += Unit_dict[k][2]
            else:
                w1m[temp_tr['game_id'].values[i]] += Unit_dict[k][0]
                w1g[temp_tr['game_id'].values[i]] += Unit_dict[k][1]
                w1c[temp_tr['game_id'].values[i]] += Unit_dict[k][2]
            
tr_u = pd.DataFrame()
tr_u['u_mine_m_0'] = w0m
tr_u['u_gas_m_0'] = w0g
tr_u['u_0'] = w0c
tr_u['u_mine_m_1'] = w1m
tr_u['u_gas_m_1'] = w1g
tr_u['u_1'] = w1c

tr_u['u_mine_m'] = tr_u['u_mine_m_0'] - tr_u['u_mine_m_1']
tr_u['u_gas_m'] = tr_u['u_gas_m_0'] - tr_u['u_gas_m_1']
tr_u['u_m'] = tr_u['u_0'] - tr_u['u_1']
    

In [ ]:
w0m = [0] * te_n
w0g = [0] * te_n
w0c = [0] * te_n # 인구수 추후 고려
w1m = [0]* te_n
w1g = [0] * te_n
w1c = [0] * te_n
for i in tqdm(range(len(temp_te))):
    for k in Unit_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            if temp_te['player'].values[i] == 0:
                w0m[temp_te['game_id'].values[i]-38872] += Unit_dict[k][0]
                w0g[temp_te['game_id'].values[i]-38872] += Unit_dict[k][1]
                w0c[temp_te['game_id'].values[i]-38872] += Unit_dict[k][2]
            else:
                w1m[temp_te['game_id'].values[i]-38872] += Unit_dict[k][0]
                w1g[temp_te['game_id'].values[i]-38872] += Unit_dict[k][1]
                w1c[temp_te['game_id'].values[i]-38872] += Unit_dict[k][2]
            
te_u = pd.DataFrame()
te_u['u_mine_m_0'] = w0m
te_u['u_gas_m_0'] = w0g
te_u['u_0'] = w0c
te_u['u_mine_m_1'] = w1m
te_u['u_gas_m_1'] = w1g
te_u['u_1'] = w1c

te_u['u_mine_m'] = te_u['u_mine_m_0'] - te_u['u_mine_m_1']
te_u['u_gas_m'] = te_u['u_gas_m_0'] - te_u['u_gas_m_1']
te_u['u_m'] = te_u['u_0'] - te_u['u_1']

# Building: extract cumulated amount

In [ ]:
# for train
w0m = [0] * tr_n
w0g = [0] * tr_n
w0c = [0] * tr_n
w1m = [0]* tr_n
w1g = [0] * tr_n
w1c = [0] * tr_n
for i in tqdm(range(len(temp_tr))):
    for k in Building_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            if temp_tr['player'].values[i] == 0:
                w0m[temp_tr['game_id'].values[i]] += Building_dict[k][0]
                w0g[temp_tr['game_id'].values[i]] += Building_dict[k][1]
                w0c[temp_tr['game_id'].values[i]] += 1
            else:
                w1m[temp_tr['game_id'].values[i]] += Building_dict[k][0]
                w1g[temp_tr['game_id'].values[i]] += Building_dict[k][1]
                w1c[temp_tr['game_id'].values[i]] += 1
            
tr_b = pd.DataFrame()
tr_b['b_mine_m_0'] = w0m
tr_b['b_gas_m_0'] = w0g
tr_b['b_0'] = w0c
tr_b['b_mine_m_1'] = w1m
tr_b['b_gas_m_1'] = w1g
tr_b['b_1'] = w1c

tr_b['b_mine_m'] = tr_b['b_mine_m_0'] - tr_b['b_mine_m_1']
tr_b['b_gas_m'] = tr_b['b_gas_m_0'] - tr_b['b_gas_m_1']
tr_b['b_m'] = tr_b['b_0'] - tr_b['b_1']
    

In [ ]:
w0m = [0] * te_n
w0g = [0] * te_n
w0c = [0] * te_n 
w1m = [0]* te_n
w1g = [0] * te_n
w1c = [0] * te_n
for i in tqdm(range(len(temp_te))):
    for k in Building_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            if temp_te['player'].values[i] == 0:
                w0m[temp_te['game_id'].values[i]-38872] += Building_dict[k][0]
                w0g[temp_te['game_id'].values[i]-38872] += Building_dict[k][1]
                w0c[temp_te['game_id'].values[i]-38872] += 1
            else:
                w1m[temp_te['game_id'].values[i]-38872] += Building_dict[k][0]
                w1g[temp_te['game_id'].values[i]-38872] += Building_dict[k][1]
                w1c[temp_te['game_id'].values[i]-38872] += 1
            
te_b = pd.DataFrame()
te_b['b_mine_m_0'] = w0m
te_b['b_gas_m_0'] = w0g
te_b['b_0'] = w0c
te_b['b_mine_m_1'] = w1m
te_b['b_gas_m_1'] = w1g
te_b['b_1'] = w1c

te_b['b_mine_m'] = te_b['b_mine_m_0'] - te_b['b_mine_m_1']
te_b['b_gas_m'] = te_b['b_gas_m_0'] - te_b['b_gas_m_1']
te_b['b_m'] = te_b['b_0'] - te_b['b_1']

# Upgrade: extract cumulated amount

In [ ]:
# for train
w0m = [0] * tr_n
w0g = [0] * tr_n
w0c = [0] * tr_n
w1m = [0]* tr_n
w1g = [0] * tr_n
w1c = [0] * tr_n
for i in tqdm(range(len(temp_tr))):
    for k in Upgrade_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            if temp_tr['player'].values[i] == 0:
                w0m[temp_tr['game_id'].values[i]] += Upgrade_dict[k][0]
                w0g[temp_tr['game_id'].values[i]] += Upgrade_dict[k][1]
                w0c[temp_tr['game_id'].values[i]] += 1
            else:
                w1m[temp_tr['game_id'].values[i]] += Upgrade_dict[k][0]
                w1g[temp_tr['game_id'].values[i]] += Upgrade_dict[k][1]
                w1c[temp_tr['game_id'].values[i]] += 1
            
tr_up = pd.DataFrame()
tr_up['up_mine_m_0'] = w0m
tr_up['up_gas_m_0'] = w0g
tr_up['up_0'] = w0c
tr_up['up_mine_m_1'] = w1m
tr_up['up_gas_m_1'] = w1g
tr_up['up_1'] = w1c

tr_up['up_mine_m'] = tr_up['up_mine_m_0'] - tr_up['up_mine_m_1']
tr_up['up_gas_m'] = tr_up['up_gas_m_0'] - tr_up['up_gas_m_1']
tr_up['up_m'] = tr_up['up_0'] - tr_up['up_1']
    

In [ ]:
w0m = [0] * te_n
w0g = [0] * te_n
w0c = [0] * te_n
w1m = [0]* te_n
w1g = [0] * te_n
w1c = [0] * te_n
for i in tqdm(range(len(temp_te))):
    for k in Upgrade_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            if temp_te['player'].values[i] == 0:
                w0m[temp_te['game_id'].values[i]-38872] += Upgrade_dict[k][0]
                w0g[temp_te['game_id'].values[i]-38872] += Upgrade_dict[k][1]
                w0c[temp_te['game_id'].values[i]-38872] += 1
            else:
                w1m[temp_te['game_id'].values[i]-38872] += Upgrade_dict[k][0]
                w1g[temp_te['game_id'].values[i]-38872] += Upgrade_dict[k][1]
                w1c[temp_te['game_id'].values[i]-38872] += 1
            
te_up = pd.DataFrame()
te_up['up_mine_m_0'] = w0m
te_up['up_gas_m_0'] = w0g
te_up['up_0'] = w0c
te_up['up_mine_m_1'] = w1m
te_up['up_gas_m_1'] = w1g
te_up['up_1'] = w1c

te_up['up_mine_m'] = te_up['up_mine_m_0'] - te_up['up_mine_m_1']
te_up['up_gas_m'] = te_up['up_gas_m_0'] - te_up['up_gas_m_1']
te_up['up_m'] = te_up['up_0'] - te_up['up_1']

# Research: extract cumulated amount

In [ ]:
# for train
w0m = [0] * tr_n
w0g = [0] * tr_n
w0c = [0] * tr_n
w1m = [0]* tr_n
w1g = [0] * tr_n
w1c = [0] * tr_n
for i in tqdm(range(len(temp_tr))):
    for k in Research_dict.keys():
        if k in temp_tr['event_contents'].values[i]:
            if temp_tr['player'].values[i] == 0:
                w0m[temp_tr['game_id'].values[i]] += Research_dict[k][0]
                w0g[temp_tr['game_id'].values[i]] += Research_dict[k][1]
                w0c[temp_tr['game_id'].values[i]] += 1
            else:
                w1m[temp_tr['game_id'].values[i]] += Research_dict[k][0]
                w1g[temp_tr['game_id'].values[i]] += Research_dict[k][1]
                w1c[temp_tr['game_id'].values[i]] += 1
            
tr_re = pd.DataFrame()
tr_re['re_mine_m_0'] = w0m
tr_re['re_gas_m_0'] = w0g
tr_re['re_0'] = w0c
tr_re['re_mine_m_1'] = w1m
tr_re['re_gas_m_1'] = w1g
tr_re['re_1'] = w1c

tr_re['re_mine_m'] = tr_re['re_mine_m_0'] - tr_re['re_mine_m_1']
tr_re['re_gas_m'] = tr_re['re_gas_m_0'] - tr_re['re_gas_m_1']
tr_re['re_m'] = tr_re['re_0'] - tr_re['re_1']
    

In [ ]:
w0m = [0] * te_n
w0g = [0] * te_n
w0c = [0] * te_n
w1m = [0]* te_n
w1g = [0] * te_n
w1c = [0] * te_n
for i in tqdm(range(len(temp_te))):
    for k in Research_dict.keys():
        if k in temp_te['event_contents'].values[i]:
            if temp_te['player'].values[i] == 0:
                w0m[temp_te['game_id'].values[i]-38872] += Research_dict[k][0]
                w0g[temp_te['game_id'].values[i]-38872] += Research_dict[k][1]
                w0c[temp_te['game_id'].values[i]-38872] += 1
            else:
                w1m[temp_te['game_id'].values[i]-38872] += Research_dict[k][0]
                w1g[temp_te['game_id'].values[i]-38872] += Research_dict[k][1]
                w1c[temp_te['game_id'].values[i]-38872] += 1
            
te_re = pd.DataFrame()
te_re['re_mine_m_0'] = w0m
te_re['re_gas_m_0'] = w0g
te_re['re_0'] = w0c
te_re['re_mine_m_1'] = w1m
te_re['re_gas_m_1'] = w1g
te_re['re_1'] = w1c

te_re['re_mine_m'] = te_re['re_mine_m_0'] - te_re['re_mine_m_1']
te_re['re_gas_m'] = te_re['re_gas_m_0'] - te_re['re_gas_m_1']
te_re['re_m'] = te_re['re_0'] - te_re['re_1']
    

In [ ]:
%%time
# attention score
tr_at = pd.DataFrame()
te_at = pd.DataFrame()
for k in tqdm(range(8)):
    tr_count = tr[tr['event'] == k].groupby(['game_id', 'player'])['event'].count()
    te_count = te[te['event'] == k].groupby(['game_id', 'player'])['event'].count()
    
    w = [np.zeros(len(tr_n)), np.zeros(len(tr_n))]
    idx = tr_count.index
    for i, j in idx:
        w[j][i] = tr_count[i][j]
    c = 'count_' + str(k)
    c1 = 'count_' + str(k) + '_dif'
    c2 = 'count_' + str(k) + '_mul'
#     c1 = 'count_' + str(k) + '_1'
#     c2 = 'count_' + str(k) + '_2'
    w0, w1 = w
    tr_at[c] = np.array(w0)-np.array(w1)

    w = [np.zeros(len(te_n)), np.zeros(len(te_n))]
    idx = te_count.index
    for i, j in idx:
        w[j][i-38872] = te_count[i][j]
    c = 'count_' + str(k)
    c1 = 'count_' + str(k) + '_dif'
    c2 = 'count_' + str(k) + '_mul'
    w0, w1 = w
    te_at[c] = np.array(w0)-np.array(w1)


In [ ]:
w0 = [0] * tr_n
w1 = [0] * tr_n
for i in tqdm(range(len(temp_tr))):
    if 'Attack' in temp_tr['event_contents'].values[i]:
        if temp_tr['player'].values[i] == 0:
            w0[temp_tr['game_id'].values[i]] += 1
        else:
            w1[temp_tr['game_id'].values[i]] += 1
            
tr_atk = pd.DataFrame(columns=['p1', 'p2'])
tr_atk['p1'] = w0
tr_atk['p2'] = w1

w0 = [0] * te_n
w1 = [0] * te_n
for i in tqdm(range(len(temp_te))):
    if 'Attack' in temp_te['event_contents'].values[i]:
        if temp_te['player'].values[i] == 0:
            w0[temp_te['game_id'].values[i]-38872] += 1
        else:
            w1[temp_te['game_id'].values[i]-38872] += 1
            
te_atk = pd.DataFrame(columns=['p1', 'p2'])
te_atk['p1'] = w0
te_atk['p2'] = w1

tr_atk['avg1'] = tr_atk['p1']/(tr_atk['p1'] + tr_atk['p2'] + 1e-10)
tr_atk['avg2'] = tr_atk['p2']/(tr_atk['p1'] + tr_atk['p2'] + 1e-10)
tr_atk['dif'] = tr_atk['avg1'] - tr_atk['avg2']

te_atk['avg1'] = te_atk['p1']/(te_atk['p1'] + te_atk['p2'] + 1e-10)
te_atk['avg2'] = te_atk['p2']/(te_atk['p1'] + te_atk['p2'] + 1e-10)
te_atk['dif'] = te_atk['avg1'] - te_atk['avg2']

tr_atk.columns = list(map(lambda x: 'atk_'+x, tr_atk.columns))
te_atk.columns = list(map(lambda x: 'atk_'+x, te_atk.columns))

In [ ]:
w0 = [0] * tr_n
w1 = [0] * tr_n
for i in tqdm(range(len(temp_tr))):
    if 'Target' in temp_tr['event_contents'].values[i]:
        if temp_tr['player'].values[i] == 0:
            w0[temp_tr['game_id'].values[i]] += 1
        else:
            w1[temp_tr['game_id'].values[i]] += 1
            
tr_tar = pd.DataFrame(columns=['p1', 'p2'])
tr_tar['p1'] = w0
tr_tar['p2'] = w1

w0 = [0] * te_n
w1 = [0] * te_n
for i in tqdm(range(len(temp_te))):
    if 'Target' in temp_te['event_contents'].values[i]:
        if temp_te['player'].values[i] == 0:
            w0[temp_te['game_id'].values[i]-38872] += 1
        else:
            w1[temp_te['game_id'].values[i]-38872] += 1
            
te_tar = pd.DataFrame(columns=['p1', 'p2'])
te_tar['p1'] = w0
te_tar['p2'] = w1

tr_tar['avg1'] = tr_tar['p1']/(tr_tar['p1'] + tr_tar['p2'] + 1e-10)
tr_tar['avg2'] = tr_tar['p2']/(tr_tar['p1'] + tr_tar['p2'] + 1e-10)
tr_tar['dif'] = tr_tar['avg1'] - tr_tar['avg2']

te_tar['avg1'] = te_tar['p1']/(te_tar['p1'] + te_tar['p2'] + 1e-10)
te_tar['avg2'] = te_tar['p2']/(te_tar['p1'] + te_tar['p2'] + 1e-10)
te_tar['dif'] = te_tar['avg1'] - te_tar['avg2']

tr_tar.columns = list(map(lambda x: 'tar_'+x, tr_tar.columns))
te_tar.columns = list(map(lambda x: 'tar_'+x, te_tar.columns))

In [ ]:
b_target = ['b_mine_m', 'u_gas_m', 'u_m']
u_target = ['u_mine_m', 'u_gas_m', 'u_m']
up_target = ['up_mine_m', 'up_gas_m', 'up_m']

In [ ]:
# tr_j = pd.DataFrame()
tr_j = pd.concat([tr_c, tr_at, tr_atk, tr_tar, tr_u, tr_b, tr_up, tr_re, tr_p], axis=1)
t

In [ ]:
tr_j

In [ ]:
############## tr
tr_j = pd.DataFrame()
tr_j['apm'] = tr_c['apm']
tr_j['apm_dif'] = tr_c['apm_p1'] - tr_c['apm_p2']

tr_j['u_mine_dif'] = tr_u['u_mine_m_0'] - tr_u['u_mine_m_1']
tr_j['u_mine_dif_min'] = (tr_u['u_mine_m_0'] - tr_u['u_mine_m_1'])/tr_time.values * 60
tr_j['u_gas_dif'] = tr_u['u_gas_m_0'] - tr_u['u_gas_m_1']
tr_j['u_gas_dif_min'] = (tr_u['u_gas_m_0'] - tr_u['u_gas_m_1'])/tr_time.values * 60
tr_j['u_dif'] = tr_u['u_0'] - tr_u['u_1']
tr_j['u_dif_min'] = (tr_u['u_0'] - tr_u['u_1'])/tr_time.values * 60

tr_j['tot_u_res'] = tr_j['u_mine_dif'] + tr_j['u_gas_dif']
tr_j['tot_u_res_min'] = tr_j['tot_u_res']/tr_time.values * 60


tr_j['b_mine_dif'] = tr_b['b_mine_m_0'] - tr_b['b_mine_m_1']
tr_j['b_mine_dif_min'] = (tr_b['b_mine_m_0'] - tr_b['b_mine_m_1'])/tr_time.values * 60
tr_j['b_gas_dif'] = tr_b['b_gas_m_0'] - tr_b['b_gas_m_1']
tr_j['b_gas_dif_min'] = (tr_b['b_gas_m_0'] - tr_b['b_gas_m_1'])/tr_time.values * 60
tr_j['b_dif'] = tr_b['b_0'] - tr_b['b_1']
tr_j['b_dif_min'] = (tr_b['b_0'] - tr_b['b_1'])/tr_time.values * 60

tr_j['tot_b_res'] = tr_j['b_mine_dif'] + tr_j['b_gas_dif']
tr_j['tot_b_res_min'] = tr_j['tot_b_res']/tr_time.values * 60


tr_j['up_mine_dif'] = tr_up['up_mine_m_0'] - tr_up['up_mine_m_1']
tr_j['up_mine_dif_min'] = (tr_up['up_mine_m_0'] - tr_up['up_mine_m_1'])/tr_time.values * 60
tr_j['up_gas_dif'] = tr_up['up_gas_m_0'] - tr_up['up_gas_m_1']
tr_j['up_gas_dif_min'] = (tr_up['up_gas_m_0'] - tr_up['up_gas_m_1'])/tr_time.values * 60
tr_j['up_dif'] = tr_up['up_0'] - tr_up['up_1']
tr_j['up_dif_min'] = (tr_up['up_0'] - tr_up['up_1'])/tr_time.values * 60

tr_j['tot_up_res'] = tr_j['up_mine_dif'] + tr_j['up_gas_dif']
tr_j['tot_up_res_min'] = tr_j['tot_up_res']/tr_time.values * 60

tr_j['re_mine_dif'] = tr_re['re_mine_m_0'] - tr_re['re_mine_m_1']
tr_j['re_mine_dif_min'] = (tr_re['re_mine_m_0'] - tr_re['re_mine_m_1'])/tr_time.values * 60
tr_j['re_gas_dif'] = tr_re['re_gas_m_0'] - tr_re['re_gas_m_1']
tr_j['re_gas_dif_min'] = (tr_re['re_gas_m_0'] - tr_re['re_gas_m_1'])/tr_time.values * 60

tr_j['re_dif'] = tr_re['re_0'] - tr_re['re_1']
tr_j['re_dif_min'] = (tr_re['re_0'] - tr_re['re_1'])/tr_time.values * 60

tr_j['tot_mine'] = tr_j['u_mine_dif'] + tr_j['b_mine_dif'] + tr_j['up_mine_dif'] + tr_j['re_mine_dif']
tr_j['tot_mine_min'] = tr_j['tot_mine']/tr_time.values * 60
tr_j['tot_gas'] = tr_j['u_gas_dif'] + tr_j['b_gas_dif'] + tr_j['up_gas_dif'] + tr_j['re_gas_dif']
tr_j['tot_gas_min'] = tr_j['tot_gas']/tr_time.values * 60


tr_j['tot_res'] = tr_j['tot_mine'] + tr_j['tot_gas']
tr_j['tot_res_min'] = tr_j['tot_res']/tr_time.values * 60

tr_j['atk_dif'] = tr_atk['atk_p1'] - tr_atk['atk_p2']
tr_j['atk_dif_avg'] = tr_atk['atk_dif']

tr_j['tar_dif'] = tr_tar['tar_p1'] - tr_tar['tar_p2']
tr_j['tar_dif_avg'] = tr_tar['tar_dif']



############# te
te_j = pd.DataFrame()
te_j['apm'] = te_c['apm']
te_j['apm_dif'] = te_c['apm_p1'] - te_c['apm_p2']

te_j['u_mine_dif'] = te_u['u_mine_m_0'] - te_u['u_mine_m_1']
te_j['u_mine_dif_min'] = (te_u['u_mine_m_0'] - te_u['u_mine_m_1'])/te_time.values * 60
te_j['u_gas_dif'] = te_u['u_gas_m_0'] - te_u['u_gas_m_1']
te_j['u_gas_dif_min'] = (te_u['u_gas_m_0'] - te_u['u_gas_m_1'])/te_time.values * 60
te_j['u_dif'] = te_u['u_0'] - te_u['u_1']
te_j['u_dif_min'] = (te_u['u_0'] - te_u['u_1'])/te_time.values * 60

te_j['tot_u_res'] = te_j['u_mine_dif'] + te_j['u_gas_dif']
te_j['tot_u_res_min'] = te_j['tot_u_res']/te_time.values * 60


te_j['b_mine_dif'] = te_b['b_mine_m_0'] - te_b['b_mine_m_1']
te_j['b_mine_dif_min'] = (te_b['b_mine_m_0'] - te_b['b_mine_m_1'])/te_time.values * 60
te_j['b_gas_dif'] = te_b['b_gas_m_0'] - te_b['b_gas_m_1']
te_j['b_gas_dif_min'] = (te_b['b_gas_m_0'] - te_b['b_gas_m_1'])/te_time.values * 60
te_j['b_dif'] = te_b['b_0'] - te_b['b_1']
te_j['b_dif_min'] = (te_b['b_0'] - te_b['b_1'])/te_time.values * 60

te_j['tot_b_res'] = te_j['b_mine_dif'] + te_j['b_gas_dif']
te_j['tot_b_res_min'] = te_j['tot_b_res']/te_time.values * 60


te_j['up_mine_dif'] = te_up['up_mine_m_0'] - te_up['up_mine_m_1']
te_j['up_mine_dif_min'] = (te_up['up_mine_m_0'] - te_up['up_mine_m_1'])/te_time.values * 60
te_j['up_gas_dif'] = te_up['up_gas_m_0'] - te_up['up_gas_m_1']
te_j['up_gas_dif_min'] = (te_up['up_gas_m_0'] - te_up['up_gas_m_1'])/te_time.values * 60
te_j['up_dif'] = te_up['up_0'] - te_up['up_1']
te_j['up_dif_min'] = (te_up['up_0'] - te_up['up_1'])/te_time.values * 60

te_j['tot_up_res'] = te_j['up_mine_dif'] + te_j['up_gas_dif']
te_j['tot_up_res_min'] = te_j['tot_up_res']/te_time.values * 60

te_j['re_mine_dif'] = te_re['re_mine_m_0'] - te_re['re_mine_m_1']
te_j['re_mine_dif_min'] = (te_re['re_mine_m_0'] - te_re['re_mine_m_1'])/te_time.values * 60
te_j['re_gas_dif'] = te_re['re_gas_m_0'] - te_re['re_gas_m_1']
te_j['re_gas_dif_min'] = (te_re['re_gas_m_0'] - te_re['re_gas_m_1'])/te_time.values * 60

te_j['re_dif'] = te_re['re_0'] - te_re['re_1']
te_j['re_dif_min'] = (te_re['re_0'] - te_re['re_1'])/te_time.values * 60

te_j['tot_mine'] = te_j['u_mine_dif'] + te_j['b_mine_dif'] + te_j['up_mine_dif'] + te_j['re_mine_dif']
te_j['tot_mine_min'] = te_j['tot_mine']/te_time.values * 60
te_j['tot_gas'] = te_j['u_gas_dif'] + te_j['b_gas_dif'] + te_j['up_gas_dif'] + te_j['re_gas_dif']
te_j['tot_gas_min'] = te_j['tot_gas']/te_time.values * 60


te_j['tot_res'] = te_j['tot_mine'] + te_j['tot_gas']
te_j['tot_res_min'] = te_j['tot_res']/te_time.values * 60

te_j['atk_dif'] = te_atk['atk_p1'] - te_atk['atk_p2']
te_j['atk_dif_avg'] = te_atk['atk_dif']

te_j['tar_dif'] = te_tar['tar_p1'] - te_tar['tar_p2']
te_j['tar_dif_avg'] = te_tar['tar_dif']

# tr_j['apm_mul'] = tr_c['apm_p1']/tr_c['apm_p2']
# tr_j['u_mine_mul'] = tr_u['u_mine_m_0'] / tr_u['u_mine_m_1']
# tr_j['u_gas_mul'] = tr_u['u_gas_m_0'] / tr_u['u_gas_m_1']
# tr_j['u_mul'] = tr_u['u_0'] / tr_u['u_1']
# tr_j['u_mine_mul'] = tr_u['u_mine_m_0'] / tr_u['u_mine_m_1']
# tr_j['u_gas_mul'] = tr_u['u_gas_m_0'] / tr_u['u_gas_m_1']
# tr_j['u_mul'] = tr_u['u_0'] / tr_u['u_1']
# tr_j['b_mine_mul'] = tr_b['b_mine_m_0'] / tr_b['b_mine_m_1']
# tr_j['b_gas_mul'] = tr_b['b_gas_m_0'] / tr_b['b_gas_m_1']
# tr_j['b_mul'] = tr_b['b_0'] / tr_b['b_1']
# tr_j['up_mine_mul'] = tr_up['up_mine_m_0'] / tr_up['up_mine_m_1']
# tr_j['up_gas_mul'] = tr_up['up_gas_m_0'] / tr_up['up_gas_m_1']
# tr_j['up_mul'] = tr_up['up_0'] / tr_up['up_1']

In [ ]:
tr_j = pd.concat([tr_j, tr_p, tr_at], axis=1)
te_j = pd.concat([te_j, te_p, te_at], axis=1)

tr_j['time'] = tr_time.values
te_j['time'] = te_time.values

In [ ]:
tr_j['match'] = (tr_j['p1'] == tr_j['p2']).map(lambda x: int(x))
te_j['match'] = (te_j['p1'] == te_j['p2']).map(lambda x: int(x))

In [ ]:
# tr_j['time'] = tr_time/60
# te_j['time'] = (te_time/60).reset_index().drop('game_id', axis=1)

In [ ]:
tr_j['wr'] = 0.00
te_j['wr'] = 0.00
for i in range(len(tr_j)):
    if tr_j['p1'].values[i] == 0 and tr_j['p2'].values[i] == 1:
        tr_j['wr'].values[i] = round(50.12-49.88, 3)
    elif tr_j['p1'].values[i] == 1 and tr_j['p2'].values[i] == 0:
        tr_j['wr'].values[i] = round(49.88-50.12, 3)
    elif tr_j['p1'].values[i] == 0 and tr_j['p2'].values[i] == 2:
        tr_j['wr'].values[i] = round(48.79-51.21, 3)
    elif tr_j['p1'].values[i] == 2 and tr_j['p2'].values[i] == 0:
        tr_j['wr'].values[i] = round(51.21-48.79, 3)
    elif tr_j['p1'].values[i] == 1 and tr_j['p2'].values[i] == 2:
        tr_j['wr'].values[i] = round(49.35-50.65, 3)
    elif tr_j['p1'].values[i] == 2 and tr_j['p2'].values[i] == 1:
        tr_j['wr'].values[i] = round(50.65-49.35, 3)
        
        
for i in range(len(te_j)):
    if te_j['p1'].values[i] == 0 and te_j['p2'].values[i] == 1:
        te_j['wr'].values[i] = round(50.12-49.88, 3)
    elif te_j['p1'].values[i] == 1 and te_j['p2'].values[i] == 0:
        te_j['wr'].values[i] = round(49.88-50.12, 3)
    elif te_j['p1'].values[i] == 0 and te_j['p2'].values[i] == 2:
        te_j['wr'].values[i] = round(48.79-51.21, 3)
    elif te_j['p1'].values[i] == 2 and te_j['p2'].values[i] == 0:
        te_j['wr'].values[i] = round(51.21-48.79, 3)
    elif te_j['p1'].values[i] == 1 and te_j['p2'].values[i] == 2:
        te_j['wr'].values[i] = round(49.35-50.65, 3)
    elif te_j['p1'].values[i] == 2 and te_j['p2'].values[i] == 1:
        te_j['wr'].values[i] = round(50.65-49.35, 3)


In [ ]:
te_j

In [ ]:
# f = open('./data/sub/tr_c.bin', 'wb')
# pk.dump(tr_c, f)
# f.close()
# f = open('./data/sub/tr_p.bin', 'wb')
# pk.dump(tr_p, f)
# f.close()
# f = open('./data/sub/tr_time.bin', 'wb')
# pk.dump(tr_time, f)
# f.close()
# f = open('./data/sub/tr_u.bin', 'wb')
# pk.dump(tr_u, f)
# f.close()
# f = open('./data/sub/tr_b.bin', 'wb')
# pk.dump(tr_b, f)
# f.close()
# f = open('./data/sub/tr_up.bin', 'wb')
# pk.dump(tr_up, f)
# f.close()
# f = open('./data/sub/tr_re.bin', 'wb')
# pk.dump(tr_re, f)
# f.close()
# f = open('./data/sub/tr_at.bin', 'wb')
# pk.dump(tr_at, f)
# f.close()
# f = open('./data/sub/tr_atk.bin', 'wb')
# pk.dump(tr_atk, f)
# f.close()
# f = open('./data/sub/tr_tar.bin', 'wb')
# pk.dump(tr_tar, f)
# f.close()
# f = open('./data/sub/te_c.bin', 'wb')
# pk.dump(te_c, f)
# f.close()
# f = open('./data/sub/te_p.bin', 'wb')
# pk.dump(te_p, f)
# f.close()
# f = open('./data/sub/te_time.bin', 'wb')
# pk.dump(te_time, f)
# f.close()
# f = open('./data/sub/te_u.bin', 'wb')
# pk.dump(te_u, f)
# f.close()
# f = open('./data/sub/te_b.bin', 'wb')
# pk.dump(te_b, f)
# f.close()
# f = open('./data/sub/te_up.bin', 'wb')
# pk.dump(te_up, f)
# f.close()
# f = open('./data/sub/te_re.bin', 'wb')
# pk.dump(te_re, f)
# f.close()
# f = open('./data/sub/te_at.bin', 'wb')
# pk.dump(te_at, f)
# f.close()
# f = open('./data/sub/te_atk.bin', 'wb')
# pk.dump(te_atk, f)
# f.close()
# f = open('./data/sub/te_tar.bin', 'wb')
# pk.dump(te_tar, f)
# f.close()

In [ ]:
f = open('./data/tr_j.bin', 'wb')
pk.dump(tr_j, f)
f.close()
f = open('./data/te_j.bin', 'wb')
pk.dump(te_j, f)
f.close()

In [ ]:
f = open('./data/tr_j.bin', 'rb')
tr_j = pk.load(f)
f.close()
f = open('./data/te_j.bin', 'rb')
te_j = pk.load(f)
f.close()


In [ ]:
############ 여기까지 저장 #######################

In [ ]:
# tr_j['t_max'] = tr_time
# te_j['t_max'] = te_time

In [ ]:
# f = open('./data/sub/tr_c.bin', 'rb')
# tr_c = pk.load(f)
# f.close()
# f = open('./data/sub/tr_p.bin', 'rb')
# tr_p = pk.load(f)
# f.close()
# f = open('./data/sub/tr_time.bin', 'rb')
# tr_time = pk.load(f)
# f.close()
# f = open('./data/sub/tr_u.bin', 'rb')
# tr_u = pk.load(f)
# f.close()
# f = open('./data/sub/tr_b.bin', 'rb')
# tr_b = pk.load(f)
# f.close()
# f = open('./data/sub/tr_up.bin', 'rb')
# tr_up = pk.load(f)
# f.close()
# f = open('./data/sub/tr_re.bin', 'rb')
# tr_re = pk.load(f)
# f.close()
# f = open('./data/sub/tr_at.bin', 'rb')
# tr_at = pk.load(f)
# f.close()
# f = open('./data/sub/tr_atk.bin', 'rb')
# tr_atk = pk.load(f)
# f.close()
# f = open('./data/sub/tr_tar.bin', 'rb')
# tr_tar = pk.load(f)
# f.close()
# f = open('./data/sub/te_c.bin', 'rb')
# te_c = pk.load(f)
# f.close()
# f = open('./data/sub/te_p.bin', 'rb')
# te_p = pk.load(f)
# f.close()
# f = open('./data/sub/te_time.bin', 'rb')
# te_time = pk.load(f)
# f.close()
# f = open('./data/sub/te_u.bin', 'rb')
# te_u = pk.load(f)
# f.close()
# f = open('./data/sub/te_b.bin', 'rb')
# te_b = pk.load(f)
# f.close()
# f = open('./data/sub/te_up.bin', 'rb')
# te_up = pk.load(f)
# f.close()
# f = open('./data/sub/te_re.bin', 'rb')
# te_re = pk.load(f)
# f.close()
# f = open('./data/sub/te_at.bin', 'rb')
# te_at = pk.load(f)
# f.close()
# f = open('./data/sub/te_atk.bin', 'rb')
# te_atk = pk.load(f)
# f.close()
# f = open('./data/sub/te_tar.bin', 'rb')
# te_tar = pk.load(f)
# f.close()